In [8]:
import tensorflow as tf
import tensorflow.python.platform.build_info as build

tf.__version__

'2.11.0'

In [9]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(f"Default GPU Device:{tf.test.gpu_device_name()}")

Num GPUs Available:  0
Default GPU Device:


2023-02-06 12:38:22.802395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-06 12:38:22.802622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-06 12:38:22.802871: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
print(build.build_info['cuda_version'])
print(build.build_info['cudnn_version'])

11.2
8
